# Run the SONAR-netCDF4 v1 convention checker on an EK60 file

In [1]:
import echopype as ep
from echopype.testing import TEST_DATA_FOLDER

from sonarnetcdf4_echopype_checker import cc

In [2]:
import datetime

print(f"echopype version: {ep.__version__}")
print(f"{datetime.datetime.utcnow()} +00:00")

echopype version: 0.7.2.dev65+ga10d22bf
2023-08-27 21:55:22.688700 +00:00


## Convert the target raw file to an EchoData object

In [3]:
ed = ep.open_raw(TEST_DATA_FOLDER / "ek60/ncei-wcsd/Summer2017-D20170620-T011027.raw", sonar_model='EK60')

# Other, contrasting files
# ed = ep.open_raw(TEST_DATA_FOLDER / "ek60/ooi/CE02SHBP-MJ01C-07-ZPLSCB101_OOI-D20191201-T000000.raw", sonar_model='EK60')

In [4]:
ed

<EchoData: standardized raw data from Internal Memory>
Top-level: contains metadata about the SONAR-netCDF4 file format.
├── Environment: contains information relevant to acoustic propagation through water.
├── Platform: contains information about the platform on which the sonar is installed.
│   └── NMEA: contains information specific to the NMEA protocol.
├── Provenance: contains metadata about how the SONAR-netCDF4 version of the data were obtained.
├── Sonar: contains sonar system metadata and sonar beam groups.
│   └── Beam_group1: contains backscatter power (uncalibrated) and other beam or channel-specific data, including split-beam angle data when they exist.
└── Vendor_specific: contains vendor-specific information about the sonar and the data.

## Global obligation settings

In [5]:
obligation_target = "M"  # "M", ['M', 'MA'], None

## Test the `Sonar/Beam_group1` group

### Read and process the CDL

In [6]:
target_group = "Sonar/Beam_group1"

In [7]:
conv_check = cc.ConventionCDL(target_group)

### Examine the target group in the converted EchoData object

In [8]:
conv_check.set_ed_group_ds(ed[target_group])

In [9]:
conv_check.ed_group_ds

<xarray.Dataset>
Dimensions:                        (channel: 3, ping_time: 1932,
                                    range_sample: 1055)
Coordinates:
  * channel                        (channel) <U37 'GPT  18 kHz 009072058c8d 1...
  * ping_time                      (ping_time) datetime64[ns] 2017-06-20T01:1...
  * range_sample                   (range_sample) int64 0 1 2 ... 1052 1053 1054
Data variables: (12/29)
    frequency_nominal              (channel) float64 1.8e+04 3.8e+04 1.2e+05
    beam_type                      (channel) int64 1 1 1
    beamwidth_twoway_alongship     (channel) float64 10.3 6.8 7.3
    beamwidth_twoway_athwartship   (channel) float64 10.3 6.8 7.2
    beam_direction_x               (channel) float64 nan nan nan
    beam_direction_y               (channel) float64 nan nan nan
    ...                             ...
    data_type                      (channel, ping_time) float32 3.0 3.0 ... 3.0
    sample_time_offset             (channel, ping_time) float64 0.0 0.0 ... 0.0
    channel_mode                   (channel, ping_time) int8 0 0 0 0 ... 0 0 0 0
    backscatter_r                  (channel, ping_time, range_sample) float32 ...
    angle_athwartship              (channel, ping_time, range_sample) float32 ...
    angle_alongship                (channel, ping_time, range_sample) float32 ...
Attributes:
    beam_mode:              vertical
    conversion_equation_t:  type_3

### Set the obligation and echopype_mods filters (optional)

In [10]:
conv_check.set_obligation(obligation_target) # "M", ['M', 'MA'], None, exclude=True

conv_check._get_obligation_vars()

array(['backscatter_r', 'beam', 'beam_direction_x', 'beam_direction_y',
       'beam_direction_z', 'beam_stabilisation', 'beam_type',
       'equivalent_beam_angle', 'non_quantitative_processing',
       'ping_time', 'sample_interval', 'sample_time_offset',
       'transmit_duration_nominal', 'transmit_frequency_start',
       'transmit_frequency_stop', 'transmit_type'], dtype=object)

`echopype_mods` handling is not fully implemented yet

In [11]:
conv_check.set_echopype_mods(None) # None

In [12]:
conv_check.echopype_mods

### Run the tests

#### Presence of expected variables

In [13]:
conv_check.test_vars_presence(test_type="expected")

****Expected variables not found in the EchoData object:
beam


#### Presence of unexpected variables

In [14]:
conv_check.test_vars_presence(test_type="unexpected")

****EchoData variables not found in the CDL:
gpt_software_version
data_type
channel_mode


#### Variable data type

Use two types of data type comparisons: strict (specific) vs generalized.

In [15]:
conv_check.test_vars_datatype(dtype_strict=True)

****Variables with different data type from what is expected:
transmit_frequency_start: EchoData type: float64, CDL type: float32
transmit_frequency_stop: EchoData type: float64, CDL type: float32
transmit_type: EchoData type: <U2, CDL type: object
sample_time_offset: EchoData type: float64, CDL type: float32
transmit_duration_nominal: EchoData type: float64, CDL type: float32


In [16]:
conv_check.test_vars_datatype(dtype_strict=False)

****Variables with different data type from what is expected:
transmit_type: EchoData type: <U2, CDL type: object


#### Variable dimensionality

In [17]:
conv_check.test_vars_dimensionality()

****EchoData variables with dimensionality different from the CDL:
beam_direction_z: EchoData dims: ('channel',), CDL dims: ('channel', 'ping_time', 'beam')
beam_direction_x: EchoData dims: ('channel',), CDL dims: ('channel', 'ping_time', 'beam')
beam_direction_y: EchoData dims: ('channel',), CDL dims: ('channel', 'ping_time', 'beam')
transmit_frequency_start: EchoData dims: ('channel',), CDL dims: ('channel', 'ping_time', 'beam')
backscatter_r: EchoData dims: ('channel', 'ping_time', 'range_sample'), CDL dims: ('channel', 'ping_time', 'range_sample', 'beam')
beam_stabilisation: EchoData dims: (), CDL dims: ('channel', 'ping_time')
transmit_frequency_stop: EchoData dims: ('channel',), CDL dims: ('channel', 'ping_time', 'beam')
beam_type: EchoData dims: ('channel',), CDL dims: ('channel', 'ping_time')
transmit_type: EchoData dims: (), CDL dims: ('channel', 'ping_time')
equivalent_beam_angle: EchoData dims: ('channel',), CDL dims: ('channel', 'ping_time', 'beam')
non_quantitative_process

#### Attribute presence

In [18]:
conv_check.test_attrs_presence()

****Variable or global missing convention attributes:
beam_stabilisation: Missing EchoData attrs: set()
transmit_type: Missing EchoData attrs: set()


In [19]:
conv_check.test_attrs_presence(global_attrs=True)

****Variable or global missing convention attributes:


#### Attribute values

In [20]:
conv_check.test_attrs_value()

****Variable or global attribute with different values:
beam_type.long_name: EchoData value: type of transducer (0-single, 1-split)
non_quantitative_processing.flag_meanings: EchoData value: ['None']


In [21]:
conv_check.test_attrs_value(global_attrs=True)

****Variable or global attribute with different values:


## Test the `Platform` group

### Read and process the CDL

In [22]:
target_group = "Platform"

In [23]:
conv_check = cc.ConventionCDL(target_group)

### Examine the target group in the converted EchoData object

In [24]:
conv_check.set_ed_group_ds(ed[target_group])

In [25]:
conv_check.ed_group_ds

<xarray.Dataset>
Dimensions:              (time1: 2933, time2: 1932, channel: 3)
Coordinates:
  * time1                (time1) datetime64[ns] 2017-06-20T01:10:27.313994752...
  * time2                (time2) datetime64[ns] 2017-06-20T01:10:27.137830912...
  * channel              (channel) <U37 'GPT  18 kHz 009072058c8d 1-1 ES18-11...
Data variables: (12/20)
    latitude             (time1) float64 44.71 44.71 44.71 ... 44.7 44.7 44.7
    longitude            (time1) float64 -124.3 -124.3 -124.3 ... -124.4 -124.4
    sentence_type        (time1) <U3 'GGA' 'GLL' 'GGA' ... 'GGA' 'GGA' 'GLL'
    pitch                (time2) float64 -0.1 -0.1 -0.1 -0.1 ... -0.1 -0.1 -0.1
    roll                 (time2) float64 -0.44 -0.44 -0.44 ... -0.44 -0.44 -0.44
    vertical_offset      (time2) float64 -0.01 -0.01 -0.01 ... -0.01 -0.01 -0.01
    ...                   ...
    position_offset_y    float64 nan
    position_offset_z    float64 nan
    transducer_offset_x  (channel) float64 0.0 0.0 0.0
    transducer_offset_y  (channel) float64 0.0 0.0 0.0
    transducer_offset_z  (channel) float64 0.0 0.0 0.0
    frequency_nominal    (channel) float64 1.8e+04 3.8e+04 1.2e+05
Attributes:
    platform_name:       
    platform_type:       
    platform_code_ICES:

### Set the obligation and echopype_mods filters (optional)

In [26]:
conv_check.set_obligation(obligation_target) # "M", ['M', 'MA'], None, exclude=True

conv_check._get_obligation_vars()

array([], dtype=object)

`echopype_mods` handling is not fully implemented yet

In [27]:
conv_check.set_echopype_mods(None) # None

In [28]:
conv_check.echopype_mods

### Run the tests

#### Presence of expected variables

In [29]:
conv_check.test_vars_presence(test_type="expected")

****Expected variables not found in the EchoData object:


#### Presence of unexpected variables

In [30]:
conv_check.test_vars_presence(test_type="unexpected")

****EchoData variables not found in the CDL:


#### Variable data type

Use two types of data type comparisons: strict (specific) vs generalized.

In [31]:
conv_check.test_vars_datatype(dtype_strict=True)

****Variables with different data type from what is expected:


In [32]:
conv_check.test_vars_datatype(dtype_strict=False)

****Variables with different data type from what is expected:


#### Variable dimensionality

In [33]:
conv_check.test_vars_dimensionality()

****EchoData variables with dimensionality different from the CDL:


#### Attribute presence

In [34]:
conv_check.test_attrs_presence()

****Variable or global missing convention attributes:


In [35]:
conv_check.test_attrs_presence(global_attrs=True)

****Variable or global missing convention attributes:


#### Attribute values

In [36]:
conv_check.test_attrs_value()

****Variable or global attribute with different values:


In [37]:
conv_check.test_attrs_value(global_attrs=True)

****Variable or global attribute with different values:
